In [2]:
import sys
from pathlib import Path

In [3]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from extracao.reading import *
from extracao.main import _format_matlab, _filter_matlab
from extracao.format import merge_close_rows, MAX_DIST
from dotenv import load_dotenv

load_dotenv()

folder = Path.cwd().parent / 'dados'


In [9]:
base = _filter_matlab(read_base(folder))

In [10]:
base

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.028,-22.662778,-43.476389,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557670,J9E,8.0
1,0.03,-18.41,-49.1,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859966,J3E,1.0
2,0.03,-22.774167,-47.004444,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859753,J3E,1.0
3,0.03,-22.926667,-43.265,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859761,J3E,0.5
4,0.03,-23.441667,-46.590833,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557823,J3E,1.0
...,...,...,...,...,...,...,...,...
817996,87.9,-20.7005555555555,-44.826388888888836,"[SRD] RADCOM, 3-A, Associacao Comunitaria Oliv...",231,631415858,<NA>,256
817997,87.9,-3.174166666666667,-41.861666666666665,"[SRD] RADCOM, P-A, Associacao De Radiodifusao ...",231,699512930,<NA>,256
817998,88.7,-19.109400000000000,-51.731600000000000,"[MOS] FM-C4, B1, Radio Patriarca De Cassilandi...",230,1004441026,<NA>,256.0
817999,97.5,-15.5671388888888333,-54.2992222222221666,"[MOS] FM-C7, B1, Sistema De Comunicacao Camarg...",230,323720978,<NA>,256.0


In [11]:
aero = read_aero(folder)

In [14]:
df1 = merge_close_rows(base.iloc[-100:], aero)

Output()

In [17]:
df2 = pd.merge(base[-100:], aero, on=['Frequency'], how='outer')

In [18]:
def distancia(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km

In [20]:
condition = np.all(df2[['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].notna(), axis=1)

In [21]:
df2.loc[condition, 'dist'] = df2.loc[condition, ['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].apply(distancia , axis=1)

In [24]:
condition = (df2.dist.notna()) & (df2.dist <= MAX_DIST) 
df2.loc[condition, 'Description_x'] = df2.loc[condition, 'Description_x'] + ' | ' + df2.loc[condition, 'Description_y']  

In [25]:
df2

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,dist
0,953.75,-19.89105,-44.81488,"[MOS] L, FB, Algar Telecom S/A (04030023541, 1...",175,1012076781,G7W,200.0,<NA>,<NA>,<NA>,NaN
1,953.75,-19.89774,-47.8412,"[MOS] L, FB, Algar Telecom S/A (04030023541, 1...",175,1012110122,G7W,200.0,<NA>,<NA>,<NA>,NaN
2,953.75,-19.90321,-44.55141,"[MOS] L, FB, Algar Telecom S/A (04030023541, 1...",175,1012074215,G7W,200.0,<NA>,<NA>,<NA>,NaN
3,953.75,-19.904,-50.7273,"[MOS] L, FB, Algar Telecom S/A (04030023541, 1...",175,1012076595,G7W,200.0,<NA>,<NA>,<NA>,NaN
4,953.75,-19.90781,-48.70027,"[MOS] L, FB, Algar Telecom S/A (04030023541, 1...",175,1012074479,G7W,200.0,<NA>,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3044,1090.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.3904800415039062,-48.481719970703125,[DOC] Radar secundário (Airbone SSR),NaN
3045,1090.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-0.9765099883079529,-62.9226188659668,[DOC] Radar secundário (Airbone SSR),NaN
3046,1090.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-0.14459000527858734,-67.05940246582031,[DOC] Radar secundário (Airbone SSR),NaN
3047,1090.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.04526999965310097,-51.0966682434082,[DOC] Radar secundário (Airbone SSR),NaN


In [ ]:
df.to_excel(folder / 'mesclados_outer_join.xlsx', index=False)

In [ ]:
aero.to_excel(folder / 'aero.xlsx', index=False)